# **GUARDRAILS**
Als we agentic AI systemen in productie willen draaien moeten we niet alleen verzekeren dat ze correcte informatie geven maar ook dat ze niet antwoorden op vragen die niet relevant zijn aan hun doel.  

Een bot die ingezet wordt voor verzekeringsadvies moet geen antwoorden kunnen geven over recepten bijvoorbeeld.  
Hiervoor kunnen we guardrail systemen gebruiken. 

In deze demo bouwen we verder op de RAG notebook en gaan we een extra laag toevoegen waarbij we via de open source guardrails-ai library het gegenereerde antwoord gaan valideren vooraleer we het terug aan de gebruiker tonen.

Je kan alle mogelijke validators binnen de guardrails-ai library [hier](https://hub.guardrailsai.com/) terugvinden.

---

## **Voorbereiding**

In [1]:
%pip install pymilvus[model] pypdf ollama --quiet

Note: you may need to restart the kernel to use updated packages.


---

## **Embedder**

In [2]:
from pymilvus import model

# Laadt een standaard embedder model
embedder = model.DefaultEmbeddingFunction()

print(f"Elke vector zal uit {embedder.dim} kommagetallen bestaan.")

/home/jens/anaconda3/envs/paai_workshop/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Elke vector zal uit 768 kommagetallen bestaan.


---

In [3]:
from pymilvus import MilvusClient

vectordb = MilvusClient("../milvus.db")

### **2. Maak een collectie aan**

In [4]:
collections = vectordb.list_collections()

if "blogpost" not in collections:
  # Aanmaken van nieuwe collectie
  vectordb.create_collection(
    collection_name="blogpost",
    dimension=embedder.dim
  )

# Lijst alle collecties op ter controle
collections = vectordb.list_collections()
print(f"Beschikbare collecties: {', '.join(collections)}")

Beschikbare collecties: LangChainCollection, blogpost


### **3. Uitlezen van PDF**

In [5]:
from pypdf import PdfReader

def lees_pdf(path):
    # Open de PDF
    pdf = PdfReader(path)
    text = ""
    # Overloop elke pagina
    for page in pdf.pages:
        # Lees de paginatekst uit
        page_text = page.extract_text()
        # Voeg de paginatekst toe aan de totale tekst
        text += page_text + "\n"
    return text

In [6]:
text = lees_pdf("Blogpost.pdf")

print(text)

Blogpost
Hoe maak je AI-modellen klein en 
efficiënt?
AI-modellen worden steeds krachtiger, maar vaak ook steeds groter. Grote 
modellen vereisen veel rekenkracht, geheugen en energie - wat ze minder 
geschikt maakt voor gebruik op edge devices, maar ook realtime computervisie 
applicaties zoals de Howest Virtual Mirror. Gelukkig zijn er diverse technieken 
om AI-modellen te verkleinen zonder de accuraatheid ervan te verlagen. In 
deze blogpost bespreken we de belangrijkste optimalisatiestrategieën: 
quantisatie, distillatie, pruning en meer. We bekijken ook welke tools je kunt 
gebruiken, op welke platformen ze draaien, en hoe groot de impact kan zijn.
1. Quantisatie
Inleiding
Binnenin een AI model zitten er miljoenen tot miljarden kommagetallen - ook 
wel “gewichtenˮ en “activatiesˮ genoemd. Elk getal neemt typisch 32 bits 
(nullen en enen) geheugen in beslag.
Voor een computer zijn kommagetallen eigenlijk zeer lastig om mee te rekenen. 
Één zoʼn berekening duurt uiteraard slechts en

### **4. Tekst opsplitsen in chunks**

In [7]:
def verdeel_in_chunks(text):
    # Kap de tekst in stukken van 512 karakters met een overlap van 128 karakters
    return [text[i:i+512] for i in range(0, len(text), 512-128)]

chunks = verdeel_in_chunks(text)

print(f"De tekst van {len(text)} karakters is opgedeeld in {len(chunks)} stukken van 512 karakters met een overlap van 128 karakters.")

De tekst van 6434 karakters is opgedeeld in 17 stukken van 512 karakters met een overlap van 128 karakters.


### **5. Chunks omzetten naar vectoren (embedding)**

In [8]:
# Omzetten van teksten naar vectoren
vectors = embedder.encode_documents(chunks)

# Een kijkje nemen naar de eerste vector = chunk 1
print(vectors[0])

[ 1.29018397e-02  3.39037056e-02  1.38407705e-02 -2.37033750e-02
  1.41432738e-02  3.95084573e-03  1.88760994e-02 -8.68979462e-03
  1.15502403e-02 -5.03735965e-02  2.82360528e-02  1.72524620e-02
 -4.01088364e-02 -9.24565788e-02  4.08351003e-03  8.70586708e-03
 -4.49316399e-02 -2.97982155e-02  9.14868732e-03 -3.51780520e-02
 -2.84741317e-02  7.10664068e-03  6.14569991e-03 -4.83656797e-02
 -1.91956371e-02  2.58579654e-02 -2.50019176e-03  6.60994206e-03
 -2.69456916e-02 -6.56962520e-03 -6.13001482e-02 -1.31405158e-02
 -1.11234524e-03 -1.92446541e-02 -5.01197160e-02 -5.89044375e-02
 -4.28711448e-03  1.35903676e-02 -1.09149342e-02  4.75778749e-02
 -1.04830449e-02  1.61411409e-02  5.04271557e-03  9.73946011e-02
 -1.69800678e-02  4.72020984e-02  3.78694586e-02 -1.64446858e-02
 -3.84101321e-02 -4.90670064e-02  5.57477220e-02 -2.05239546e-02
 -1.50153661e-02  2.02639093e-02 -3.07706504e-02  1.58992888e-02
  2.00894465e-02 -5.43083171e-02  1.18508442e-02 -2.68718704e-02
  1.21450267e-02 -2.39214

### **6. Vectoren in database stoppen**

In [9]:
from pymilvus import MilvusClient

client = MilvusClient("../milvus.db")

# Formatteer de vectoren als een lijst van dictionaries
data = [ {"text": text, "vector": vector, "id": id} for id, (text, vector) in enumerate(zip(chunks, vectors)) ]

# Vectoren toevoegen aan de blogpost collectie
client.insert("blogpost", data)

{'insert_count': 17, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], 'cost': 0}

---

## **Vector database bevragen** *(= Retrieval)*

### **6. Vraag embedden**

In [10]:
question = "Wat kan je me vertellen over de blogpost?"

question_vector = embedder.encode_queries([question])[0]

### **7. Relevante documenten zoeken**

In [11]:
from pymilvus import MilvusClient

client = MilvusClient("../milvus.db")

results = client.search(
    collection_name="blogpost",
    data=[question_vector],
    output_fields=["text"]
)

print("Resultaten:")
for result in results[0]:
    print(result)

Resultaten:
{'id': 0, 'distance': 0.43110722303390503, 'entity': {'text': 'Blogpost\nHoe maak je AI-modellen klein en \nefficiënt?\nAI-modellen worden steeds krachtiger, maar vaak ook steeds groter. Grote \nmodellen vereisen veel rekenkracht, geheugen en energie - wat ze minder \ngeschikt maakt voor gebruik op edge devices, maar ook realtime computervisie \napplicaties zoals de Howest Virtual Mirror. Gelukkig zijn er diverse technieken \nom AI-modellen te verkleinen zonder de accuraatheid ervan te verlagen. In \ndeze blogpost bespreken we de belangrijkste optimalisatiestrategieën: \nquanti'}}
{'id': 1, 'distance': 0.4102115035057068, 'entity': {'text': 'inen zonder de accuraatheid ervan te verlagen. In \ndeze blogpost bespreken we de belangrijkste optimalisatiestrategieën: \nquantisatie, distillatie, pruning en meer. We bekijken ook welke tools je kunt \ngebruiken, op welke platformen ze draaien, en hoe groot de impact kan zijn.\n1. Quantisatie\nInleiding\nBinnenin een AI model zitten 

---

## **Antwoord formuleren** *(= Generation)*

### **8. Taalmodel bevragen**

In [12]:
from ollama import chat

prompt_template = """
Je bent een professionele assistent. Gebruik onderstaande context om de vraag te beantwoorden.
Als het antwoord niet in de context staat, zeg dan dat je het niet weet.

### Context:
{context}

### Vraag:
{question}

### Antwoord:
"""

def vraag_ollama_rag(context, question, model="llama3"):
    response = chat(model=model, messages=[{"role": "user", "content": prompt_template.format(context=context, question=question)}])
    return response.message.content.strip()

# Chunks van gevonden documenten terug aan elkaar plakken om context te vormen
context = "\n\n".join([result.entity.text.strip() for result in results[0]])

# Vraag stellen aan Ollama met de context en de vraag
answer = vraag_ollama_rag(context, question)

print(answer)

Deze blogpost behandelt hoe AI-modellen kunnen worden gemaakt en geoptimaliseerd om kleiner en efficiënter te zijn. De focus ligt op verschillende optimalisatiestrategieën zoals quantisatie, distillatie, pruning en meer. Het doel is om de accuraatheid van het model te behouden terwijl het gebruik van resources wordt verminderd.


---

## **Verificatie**

### **9. Installeer guardrails-ai**

In [13]:
%pip install guardrails-ai
!guardrails configure
!guardrails hub install hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached arrow-1.3.0-py3-none-any.whl.metadata (7.5 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 11.2 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 11.1 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.4/948.4 kB 8.6 MB/s  0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.2 MB/s  0:00:00
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)
Using cached arrow-1.3.0-py3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


E0000 00:00:1758632508.834696  133009 backup_poller.cc:139] Run client channel backup poller: UNKNOWN:pollset_work {children:[UNKNOWN:epoll_wait: Bad file descriptor (9)]}
Enable anonymous metrics reporting? [Y/n]: ^C
Aborted.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR:guardrails-cli:Invalid URI! The package URI must start with 'hub://'


### **10. Installeer de RestrictToTopic validator**

In [ ]:
!guardrails hub install hub://tryolabs/restricttotopic

### **11. Maak een Guard object aan**

from guardrails import Guard
from guardrails.hub import RestrictToTopic

guard = Guard().use(
    RestrictToTopic(
        valid_topics=["AI"],
        invalid_topics=["sports"]
        disable_classifier=True,
        disable_llm=False,
        on_fail="filter"
    )
)

### **12. Gebruik de guard om response van het model te valideren** 

In [ ]:
validation_step = guard.validate(answer)

# Print validated response
if validation_step.validation_passed:
    print(response)
else:
    print("Validation Failed", validation_step.validation_summaries[0].failure_reason)